# �� **Módulo 6: Retrieval - Encontrando as Informações Certas**

> *Como um detetive que usa várias pistas para resolver um caso*

---

## **Aula 6.1: Retrieval Básico - Busca por Similaridade**

---

### **Tá, mas o que é Retrieval?**

Retrieval é como ter um **detetive super inteligente** que, quando você faz uma pergunta, vai buscar nos documentos e encontra exatamente as informações que você precisa. É a parte do RAG que "encontra as pistas certas".

**🖼️ Sugestão de imagem**: Um detetive com lupa analisando documentos

### **Setup Inicial - Preparando o Terreno**

**⚠️ IMPORTANTE**: Se você não executou os notebooks anteriores, execute a célula abaixo para instalar as dependências.

In [ ]:
# 🚀 SETUP GRATUITO PARA COLAB
# Execute esta célula primeiro para configurar o ambiente!

# Instalando dependências (execute apenas se necessário)
!pip install langchain>=0.1.0
!pip install langchain-community>=0.0.10
!pip install langchain-core>=0.1.0
!pip install python-dotenv>=1.0.0
!pip install sentence-transformers>=2.2.0
!pip install chromadb>=0.4.0
!pip install faiss-cpu>=1.7.0
!pip install numpy>=1.24.0
!pip install scikit-learn>=1.3.0

print("✅ Dependências instaladas com sucesso!")
print("🔍 Ambiente configurado para Retrieval!")

In [ ]:
# 🔍 IMPORTAÇÕES PARA RETRIEVAL
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json

print("✅ Componentes de Retrieval importados com sucesso!")
print("🔍 Pronto para encontrar informações!")

### **Exemplo Prático - Criando Base de Conhecimento**

Vamos criar uma base de conhecimento sobre tecnologia para demonstrar o retrieval:

In [ ]:
# 🧪 EXEMPLO PRÁTICO: BASE DE CONHECIMENTO SOBRE TECNOLOGIA

# Documentos sobre diferentes tecnologias
documentos_tech = [
    {
        "titulo": "Python para Iniciantes",
        "conteudo": """
        Python é uma linguagem de programação de alto nível, interpretada e de propósito geral. 
        Foi criada por Guido van Rossum e lançada em 1991. Python é conhecida por sua sintaxe 
        simples e legível, que enfatiza a legibilidade do código.
        
        Características principais:
        - Sintaxe limpa e legível
        - Grande biblioteca padrão
        - Suporte a múltiplos paradigmas
        - Ideal para iniciantes
        - Muito usado em data science e IA
        """
    },
    {
        "titulo": "Machine Learning Básico",
        "conteudo": """
        Machine Learning é um subcampo da inteligência artificial que se concentra no 
        desenvolvimento de algoritmos que podem aprender e fazer previsões baseadas em dados.
        
        Tipos principais:
        - Supervised Learning: Aprende com dados rotulados
        - Unsupervised Learning: Encontra padrões em dados não rotulados
        - Reinforcement Learning: Aprende através de tentativa e erro
        
        Aplicações comuns:
        - Reconhecimento de imagem
        - Processamento de linguagem natural
        - Sistemas de recomendação
        - Análise preditiva
        """
    },
    {
        "titulo": "Desenvolvimento Web com React",
        "conteudo": """
        React é uma biblioteca JavaScript criada pelo Facebook para construir interfaces 
        de usuário. É baseada em componentes reutilizáveis e usa um DOM virtual para 
        otimizar performance.
        
        Conceitos principais:
        - Componentes funcionais e de classe
        - Props e State
        - Hooks (useState, useEffect)
        - JSX para renderização
        - Virtual DOM para performance
        
        Vantagens:
        - Grande ecossistema
        - Comunidade ativa
        - Reutilização de componentes
        - Performance otimizada
        """
    },
    {
        "titulo": "Banco de Dados SQL",
        "conteudo": """
        SQL (Structured Query Language) é uma linguagem padrão para gerenciar bancos de 
        dados relacionais. Permite criar, modificar e consultar dados de forma estruturada.
        
        Comandos principais:
        - SELECT: Consultar dados
        - INSERT: Inserir novos registros
        - UPDATE: Modificar dados existentes
        - DELETE: Remover registros
        - CREATE: Criar tabelas e estruturas
        
        Conceitos importantes:
        - Chaves primárias e estrangeiras
        - Índices para performance
        - Normalização de dados
        - Transações ACID
        """
    },
    {
        "titulo": "DevOps e CI/CD",
        "conteudo": """
        DevOps é uma cultura que combina desenvolvimento de software (Dev) com operações 
        de TI (Ops). CI/CD (Continuous Integration/Continuous Deployment) é uma prática 
        que automatiza o processo de desenvolvimento e entrega.
        
        Ferramentas populares:
        - Jenkins para automação
        - Docker para containerização
        - Kubernetes para orquestração
        - Git para controle de versão
        - Terraform para infraestrutura como código
        
        Benefícios:
        - Entrega mais rápida
        - Maior qualidade de código
        - Redução de erros
        - Melhor colaboração
        """
    }
]

print("✅ Base de conhecimento criada com sucesso!")
print(f"📚 {len(documentos_tech)} documentos sobre tecnologia")

### **1. Preparando os Documentos para Retrieval**

Primeiro, vamos processar os documentos e criar embeddings:

In [ ]:
# 🔧 PREPARANDO DOCUMENTOS PARA RETRIEVAL

# Criando embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Processando documentos
documentos_processados = []
for doc in documentos_tech:
    # Dividindo o conteúdo em chunks menores
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50
    )
    
    chunks = splitter.split_text(doc["conteudo"])
    
    # Criando objetos Document
    for i, chunk in enumerate(chunks):
        documentos_processados.append(
            Document(
                page_content=chunk,
                metadata={
                    "titulo": doc["titulo"],
                    "chunk_id": i,
                    "tipo": "tecnologia"
                }
            )
        )

print(f"✅ Documentos processados: {len(documentos_processados)} chunks")
print(f"📏 Tamanho médio dos chunks: {sum(len(doc.page_content) for doc in documentos_processados) / len(documentos_processados):.0f} caracteres")

### **2. Criando Vector Store com Chroma**

Vamos usar Chroma como nossa primeira vector store:

In [ ]:
# ��️ CRIANDO VECTOR STORE COM CHROMA

# Criando vector store
vectorstore_chroma = Chroma.from_documents(
    documents=documentos_processados,
    embedding=embeddings,
    collection_name="tech_knowledge"
)

print("✅ Vector Store Chroma criado com sucesso!")
print(f"📊 Documentos indexados: {vectorstore_chroma._collection.count()}")

# Testando busca simples
query = "Como aprender Python?"
resultados = vectorstore_chroma.similarity_search(query, k=3)

print(f"\n🔍 Resultados para: '{query}'")
for i, doc in enumerate(resultados):
    print(f"\n�� Resultado {i+1}:")
    print(f"Título: {doc.metadata['titulo']}")
    print(f"Conteúdo: {doc.page_content[:200]}...")
    print(f"Similaridade: {doc.metadata.get('score', 'N/A')}")

### **3. Busca por Similaridade - O Método Mais Comum**

Vamos explorar diferentes tipos de busca por similaridade:

In [ ]:
# 🔍 BUSCA POR SIMILARIDADE - MÉTODOS DIFERENTES

# Lista de queries para testar
queries_teste = [
    "Como começar a programar?",
    "O que é machine learning?",
    "Ferramentas de desenvolvimento web",
    "Como gerenciar bancos de dados?",
    "Automação de deploy"
]

print("🔍 TESTANDO DIFERENTES QUERIES:")
print("=" * 50)

for query in queries_teste:
    print(f"\n❓ Query: '{query}'")
    
    # Busca por similaridade
    resultados = vectorstore_chroma.similarity_search(query, k=2)
    
    for i, doc in enumerate(resultados):
        print(f"   📄 {i+1}. {doc.metadata['titulo']}")
        print(f"      {doc.page_content[:100]}...")
    
    print("-" * 30)

# Busca com score de similaridade
print("\n📊 BUSCA COM SCORES DE SIMILARIDADE:")
query_exemplo = "Linguagem de programação para iniciantes"
resultados_com_score = vectorstore_chroma.similarity_search_with_score(query_exemplo, k=3)

for i, (doc, score) in enumerate(resultados_com_score):
    print(f"\n📄 Resultado {i+1} (Score: {score:.4f}):")
    print(f"Título: {doc.metadata['titulo']}")
    print(f"Conteúdo: {doc.page_content[:150]}...")

### **4. Busca com Filtros - Encontrando Informações Específicas**

Vamos aprender a usar filtros para refinar nossas buscas:

In [ ]:
# 🎯 BUSCA COM FILTROS - REFINANDO RESULTADOS

# Busca com filtro por título
print("🎯 BUSCA COM FILTRO POR TÍTULO:")
query = "programação"
resultados_filtrados = vectorstore_chroma.similarity_search(
    query,
    k=5,
    filter={"titulo": {"$contains": "Python"}}
)

print(f"Query: '{query}' (filtro: título contém 'Python')")
for i, doc in enumerate(resultados_filtrados):
    print(f"   📄 {i+1}. {doc.metadata['titulo']}")
    print(f"      {doc.page_content[:100]}...")

# Busca com múltiplos filtros
print("\n🎯 BUSCA COM MÚLTIPLOS FILTROS:")
resultados_multiplos = vectorstore_chroma.similarity_search(
    "desenvolvimento",
    k=3,
    filter={
        "$or": [
            {"titulo": {"$contains": "React"}},
            {"titulo": {"$contains": "Web"}}
        ]
    }
)

print(f"Query: 'desenvolvimento' (filtro: React OU Web)")
for i, doc in enumerate(resultados_multiplos):
    print(f"   📄 {i+1}. {doc.metadata['titulo']}")
    print(f"      {doc.page_content[:100]}...")

# Busca por tipo de documento
print("\n🎯 BUSCA POR TIPO DE DOCUMENTO:")
resultados_tipo = vectorstore_chroma.similarity_search(
    "aprendizado",
    k=3,
    filter={"tipo": "tecnologia"}
)

print(f"Query: 'aprendizado' (filtro: tipo = tecnologia)")
for i, doc in enumerate(resultados_tipo):
    print(f"   📄 {i+1}. {doc.metadata['titulo']}")
    print(f"      {doc.page_content[:100]}...")

---

## **Aula 6.2: Retrieval Avançado - Múltiplas Estratégias**

---

### **Por que precisamos de estratégias avançadas?**

Imagine que você é um detetive que não usa apenas uma pista, mas **combina várias estratégias** para resolver um caso. No retrieval avançado, usamos múltiplas técnicas para encontrar as melhores informações.

**🖼️ Sugestão de imagem**: Detetive usando múltiplas ferramentas de investigação

### **1. Multi-Query Retrieval - Várias Perguntas, Uma Resposta**

Vamos implementar uma estratégia que faz várias perguntas relacionadas:

In [ ]:
# 🔄 MULTI-QUERY RETRIEVAL - VÁRIAS PERGUNTAS, UMA RESPOSTA

def gerar_queries_relacionadas(query_principal):
    """
    Gera queries relacionadas para melhorar o retrieval
    """
    queries = [query_principal]
    
    # Adicionando variações da query principal
    if "como" in query_principal.lower():
        queries.append(query_principal.replace("como", "o que é"))
        queries.append(query_principal.replace("como", "quando usar"))
    
    if "melhor" in query_principal.lower():
        queries.append(query_principal.replace("melhor", "top"))
        queries.append(query_principal.replace("melhor", "recomendado"))
    
    # Adicionando queries por sinônimos
    sinônimos = {
        "programar": ["codificar", "desenvolver", "criar código"],
        "aprender": ["estudar", "entender", "dominar"],
        "ferramenta": ["software", "aplicação", "programa"],
        "banco de dados": ["database", "db", "sql"],
        "web": ["internet", "site", "aplicação web"]
    }
    
    for palavra, sinonimos in sinônimos.items():
        if palavra in query_principal.lower():
            for sinonimo in sinonimos:
                nova_query = query_principal.lower().replace(palavra, sinonimo)
                if nova_query not in queries:
                    queries.append(nova_query)
    
    return queries[:5]  # Limitando a 5 queries

def multi_query_retrieval(query_principal, vectorstore, k=3):
    """
    Executa retrieval com múltiplas queries relacionadas
    """
    queries = gerar_queries_relacionadas(query_principal)
    
    print(f"�� Multi-Query Retrieval para: '{query_principal}'")
    print(f"�� Queries geradas: {queries}")
    
    todos_resultados = []
    
    for query in queries:
        resultados = vectorstore.similarity_search_with_score(query, k=k)
        for doc, score in resultados:
            todos_resultados.append((doc, score, query))
    
    # Removendo duplicatas e ordenando por score
    resultados_unicos = {}
    for doc, score, query_origem in todos_resultados:
        doc_id = f"{doc.metadata['titulo']}_{doc.metadata['chunk_id']}"
        if doc_id not in resultados_unicos or score > resultados_unicos[doc_id][1]:
            resultados_unicos[doc_id] = (doc, score, query_origem)
    
    # Ordenando por score (melhor primeiro)
    resultados_finais = sorted(
        resultados_unicos.values(),
        key=lambda x: x[1],
        reverse=True
    )
    
    return resultados_finais[:k]

# Testando multi-query retrieval
query_teste = "Como aprender a programar?"
resultados_multi = multi_query_retrieval(query_teste, vectorstore_chroma, k=3)

print(f"\n🏆 MELHORES RESULTADOS MULTI-QUERY:")
for i, (doc, score, query_origem) in enumerate(resultados_multi):
    print(f"\n📄 Resultado {i+1} (Score: {score:.4f}):")
    print(f"Título: {doc.metadata['titulo']}")
    print(f"Query origem: '{query_origem}'")
    print(f"Conteúdo: {doc.page_content[:150]}...")

### **2. Re-ranking - Melhorando a Qualidade dos Resultados**

Vamos implementar um sistema de re-ranking para melhorar a qualidade:

In [ ]:
# 📊 RE-RANKING - MELHORANDO A QUALIDADE DOS RESULTADOS

def calcular_relevancia_semantica(query, documento):
    """
    Calcula relevância semântica entre query e documento
    """
    # Palavras-chave importantes
    palavras_chave = {
        "programar": ["código", "linguagem", "sintaxe", "desenvolver"],
        "aprender": ["estudar", "tutorial", "iniciante", "básico"],
        "machine learning": ["algoritmo", "dados", "previsão", "modelo"],
        "web": ["react", "javascript", "frontend", "interface"],
        "banco de dados": ["sql", "tabela", "query", "dados"],
        "devops": ["deploy", "automação", "ci/cd", "infraestrutura"]
    }
    
    score = 0
    query_lower = query.lower()
    doc_lower = documento.lower()
    
    # Verificando palavras-chave
    for categoria, palavras in palavras_chave.items():
        if categoria in query_lower:
            for palavra in palavras:
                if palavra in doc_lower:
                    score += 1
    
    # Verificando sobreposição de palavras
    palavras_query = set(query_lower.split())
    palavras_doc = set(doc_lower.split())
    sobreposicao = len(palavras_query.intersection(palavras_doc))
    score += sobreposicao * 0.5
    
    return score

def re_rank_resultados(query, resultados_com_score):
    """
    Re-rankeia resultados baseado em múltiplos critérios
    """
    resultados_re_ranked = []
    
    for doc, score_original in resultados_com_score:
        # Score de similaridade original (normalizado)
        score_similaridade = 1 - score_original  # Invertendo (maior = melhor)
        
        # Score de relevância semântica
        score_semantico = calcular_relevancia_semantica(query, doc.page_content)
        
        # Score de qualidade do documento
        score_qualidade = 0
        if len(doc.page_content) > 100:  # Documentos mais longos
            score_qualidade += 0.2
        if doc.metadata.get('titulo', '').lower() in query.lower():
            score_qualidade += 0.3
        
        # Score final combinado
        score_final = (
            score_similaridade * 0.5 +
            score_semantico * 0.3 +
            score_qualidade * 0.2
        )
        
        resultados_re_ranked.append((doc, score_final, score_original))
    
    # Ordenando por score final
    resultados_re_ranked.sort(key=lambda x: x[1], reverse=True)
    
    return resultados_re_ranked

# Testando re-ranking
query_teste = "Como aprender Python para iniciantes?"
resultados_originais = vectorstore_chroma.similarity_search_with_score(query_teste, k=5)

print("📊 COMPARAÇÃO: ORIGINAL vs RE-RANKED")
print("=" * 50)

print("\n�� RESULTADOS ORIGINAIS:")
for i, (doc, score) in enumerate(resultados_originais):
    print(f"{i+1}. {doc.metadata['titulo']} (Score: {score:.4f})")

print("\n�� RESULTADOS RE-RANKED:")
resultados_re_ranked = re_rank_resultados(query_teste, resultados_originais)
for i, (doc, score_final, score_original) in enumerate(resultados_re_ranked):
    print(f"{i+1}. {doc.metadata['titulo']} (Final: {score_final:.4f}, Original: {score_original:.4f})")

### **3. Retrieval Híbrido - Combinando Diferentes Estratégias**

Vamos criar um sistema que combina busca por similaridade com busca por palavras-chave:

In [ ]:
# 🔀 RETRIEVAL HÍBRIDO - COMBINANDO ESTRATÉGIAS

def busca_por_palavras_chave(query, documentos, k=3):
    """
    Busca simples por palavras-chave
    """
    palavras_query = set(query.lower().split())
    resultados = []
    
    for doc in documentos:
        palavras_doc = set(doc.page_content.lower().split())
        sobreposicao = len(palavras_query.intersection(palavras_doc))
        
        if sobreposicao > 0:
            score = sobreposicao / len(palavras_query)
            resultados.append((doc, score))
    
    # Ordenando por score
    resultados.sort(key=lambda x: x[1], reverse=True)
    return resultados[:k]

def retrieval_hibrido(query, vectorstore, documentos, k=3, peso_similaridade=0.7):
    """
    Combina busca por similaridade com busca por palavras-chave
    """
    # Busca por similaridade
    resultados_similaridade = vectorstore.similarity_search_with_score(query, k=k*2)
    
    # Busca por palavras-chave
    resultados_keywords = busca_por_palavras_chave(query, documentos, k=k*2)
    
    # Combinando resultados
    resultados_combinados = {}
    
    # Adicionando resultados de similaridade
    for doc, score_sim in resultados_similaridade:
        doc_id = f"{doc.metadata['titulo']}_{doc.metadata['chunk_id']}"
        score_normalizado = 1 - score_sim  # Invertendo score
        resultados_combinados[doc_id] = {
            'doc': doc,
            'score_similaridade': score_normalizado,
            'score_keywords': 0,
            'score_final': score_normalizado * peso_similaridade
        }
    
    # Adicionando resultados de palavras-chave
    for doc, score_key in resultados_keywords:
        doc_id = f"{doc.metadata['titulo']}_{doc.metadata['chunk_id']}"
        if doc_id in resultados_combinados:
            resultados_combinados[doc_id]['score_keywords'] = score_key
            resultados_combinados[doc_id]['score_final'] += score_key * (1 - peso_similaridade)
        else:
            resultados_combinados[doc_id] = {
                'doc': doc,
                'score_similaridade': 0,
                'score_keywords': score_key,
                'score_final': score_key * (1 - peso_similaridade)
            }
    
    # Ordenando por score final
    resultados_finais = sorted(
        resultados_combinados.values(),
        key=lambda x: x['score_final'],
        reverse=True
    )
    
    return resultados_finais[:k]

# Testando retrieval híbrido
query_teste = "Ferramentas para desenvolvimento web"
resultados_hibrido = retrieval_hibrido(
    query_teste, 
    vectorstore_chroma, 
    documentos_processados, 
    k=3
)

print("🔀 RETRIEVAL HÍBRIDO:")
print(f"Query: '{query_teste}'")
print("=" * 50)

for i, resultado in enumerate(resultados_hibrido):
    print(f"\n�� Resultado {i+1}:")
    print(f"Título: {resultado['doc'].metadata['titulo']}")
    print(f"Score Similaridade: {resultado['score_similaridade']:.4f}")
    print(f"Score Keywords: {resultado['score_keywords']:.4f}")
    print(f"Score Final: {resultado['score_final']:.4f}")
    print(f"Conteúdo: {resultado['doc'].page_content[:150]}...")

### **4. Avaliação de Qualidade do Retrieval**

Vamos criar métricas para avaliar a qualidade do nosso sistema de retrieval:

In [ ]:
# 📊 AVALIAÇÃO DE QUALIDADE DO RETRIEVAL

def avaliar_retrieval(query, resultados, relevancia_esperada):
    """
    Avalia a qualidade do retrieval
    """
    # Métricas básicas
    precision = 0
    recall = 0
    
    # Verificando relevância dos resultados
    resultados_relevantes = 0
    for resultado in resultados:
        titulo = resultado['doc'].metadata['titulo'].lower()
        if any(palavra in titulo for palavra in relevancia_esperada):
            resultados_relevantes += 1
    
    precision = resultados_relevantes / len(resultados) if resultados else 0
    recall = resultados_relevantes / len(relevancia_esperada) if relevancia_esperada else 0
    
    # F1-Score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'resultados_relevantes': resultados_relevantes,
        'total_resultados': len(resultados)
    }

# Testando avaliação
queries_teste = [
    {
        'query': 'Como aprender Python?',
        'relevancia_esperada': ['python', 'iniciantes', 'aprender']
    },
    {
        'query': 'Ferramentas de desenvolvimento web',
        'relevancia_esperada': ['react', 'web', 'desenvolvimento']
    },
    {
        'query': 'Machine learning para iniciantes',
        'relevancia_esperada': ['machine learning', 'algoritmo', 'dados']
    }
]

print("📊 AVALIAÇÃO DE QUALIDADE DO RETRIEVAL:")
print("=" * 50)

for teste in queries_teste:
    print(f"\n🔍 Query: '{teste['query']}'")
    
    # Executando retrieval
    resultados = retrieval_hibrido(
        teste['query'], 
        vectorstore_chroma, 
        documentos_processados, 
        k=3
    )
    
    # Avaliando qualidade
    metricas = avaliar_retrieval(teste['query'], resultados, teste['relevancia_esperada'])
    
    print(f"   �� Precision: {metricas['precision']:.3f}")
    print(f"   📊 Recall: {metricas['recall']:.3f}")
    print(f"   📊 F1-Score: {metricas['f1_score']:.3f}")
    print(f"   📄 Resultados relevantes: {metricas['resultados_relevantes']}/{metricas['total_resultados']}")
    
    # Mostrando resultados
    for i, resultado in enumerate(resultados):
        titulo = resultado['doc'].metadata['titulo']
        relevante = any(palavra in titulo.lower() for palavra in teste['relevancia_esperada'])
        status = "✅" if relevante else "❌"
        print(f"      {status} {i+1}. {titulo}")

### **5. Exemplo Prático: Sistema de Busca de Documentação**

Vamos criar um sistema completo de busca de documentação:

In [ ]:
# �� EXEMPLO PRÁTICO: SISTEMA DE BUSCA DE DOCUMENTAÇÃO

class SistemaBuscaDocumentacao:
    def __init__(self, vectorstore, documentos):
        self.vectorstore = vectorstore
        self.documentos = documentos
        self.historico_buscas = []
    
    def buscar(self, query, k=3, estrategia='hibrido'):
        """
        Executa busca com diferentes estratégias
        """
        if estrategia == 'similaridade':
            resultados = self.vectorstore.similarity_search_with_score(query, k=k)
            return [(doc, score, 'similaridade') for doc, score in resultados]
        
        elif estrategia == 'keywords':
            resultados = busca_por_palavras_chave(query, self.documentos, k=k)
            return [(doc, score, 'keywords') for doc, score in resultados]
        
        elif estrategia == 'hibrido':
            resultados = retrieval_hibrido(query, self.vectorstore, self.documentos, k=k)
            return [(resultado['doc'], resultado['score_final'], 'hibrido') for resultado in resultados]
        
        elif estrategia == 'multi_query':
            resultados = multi_query_retrieval(query, self.vectorstore, k=k)
            return [(doc, score, 'multi_query') for doc, score, _ in resultados]
    
    def buscar_avancada(self, query, filtros=None, k=3):
        """
        Busca avançada com filtros
        """
        if filtros:
            resultados = self.vectorstore.similarity_search(query, k=k*2, filter=filtros)
        else:
            resultados = self.vectorstore.similarity_search(query, k=k*2)
        
        # Re-ranking dos resultados
        resultados_com_score = [(doc, 0.0) for doc in resultados]  # Score placeholder
        resultados_re_ranked = re_rank_resultados(query, resultados_com_score)
        
        return resultados_re_ranked[:k]
    
    def adicionar_ao_historico(self, query, resultados):
        """
        Adiciona busca ao histórico
        """
        self.historico_buscas.append({
            'query': query,
            'resultados': len(resultados),
            'timestamp': '2024-01-15'  # Simplificado
        })
    
    def sugerir_queries(self, query_atual):
        """
        Sugere queries relacionadas
        """
        sugestoes = []
        
        # Baseado em palavras-chave
        if 'python' in query_atual.lower():
            sugestoes.extend(['Como instalar Python?', 'Melhores IDEs para Python', 'Python vs JavaScript'])
        
        if 'machine learning' in query_atual.lower():
            sugestoes.extend(['Frameworks de ML', 'Como começar com ML', 'Aplicações de ML'])
        
        if 'web' in query_atual.lower():
            sugestoes.extend(['Frameworks web', 'Frontend vs Backend', 'APIs REST'])
        
        return sugestoes[:3]

# Criando e testando o sistema
sistema_busca = SistemaBuscaDocumentacao(vectorstore_chroma, documentos_processados)

print("📚 SISTEMA DE BUSCA DE DOCUMENTAÇÃO")
print("=" * 50)

# Testando diferentes estratégias
query_teste = "Como aprender programação?"

estrategias = ['similaridade', 'keywords', 'hibrido', 'multi_query']

for estrategia in estrategias:
    print(f"\n🔍 Estratégia: {estrategia.upper()}")
    resultados = sistema_busca.buscar(query_teste, k=2, estrategia=estrategia)
    
    for i, (doc, score, tipo) in enumerate(resultados):
        print(f"   📄 {i+1}. {doc.metadata['titulo']} (Score: {score:.4f})")

# Testando busca avançada
print(f"\n🔍 BUSCA AVANÇADA:")
resultados_avancada = sistema_busca.buscar_avancada(
    "desenvolvimento web",
    filtros={"titulo": {"$contains": "React"}},
    k=2
)

for i, (doc, score_final, score_original) in enumerate(resultados_avancada):
    print(f"   📄 {i+1}. {doc.metadata['titulo']} (Score: {score_final:.4f})")

# Sugestões de queries
print(f"\n💡 SUGESTÕES DE QUERIES:")
sugestoes = sistema_busca.sugerir_queries("Python para iniciantes")
for i, sugestao in enumerate(sugestoes):
    print(f"   �� {i+1}. {sugestao}")

### **6. Exercício Prático**

Agora é sua vez! Vamos criar um sistema de retrieval personalizado:

In [ ]:
# �� EXERCÍCIO PRÁTICO: RETRIEVAL PERSONALIZADO

# 1. Crie uma nova base de conhecimento sobre um assunto de sua escolha
print("�� EXERCÍCIO 1: CRIAR BASE DE CONHECIMENTO")
# Sua implementação aqui...

# 2. Implemente uma estratégia de retrieval personalizada
print("\n🎯 EXERCÍCIO 2: ESTRATÉGIA PERSONALIZADA")
# Sua implementação aqui...

# 3. Compare diferentes estratégias de retrieval
print("\n🎯 EXERCÍCIO 3: COMPARAÇÃO DE ESTRATÉGIAS")
# Sua implementação aqui...

# 4. Avalie a qualidade do seu sistema
print("\n🎯 EXERCÍCIO 4: AVALIAÇÃO DE QUALIDADE")
# Sua implementação aqui...

# 5. Implemente uma funcionalidade adicional
print("\n🎯 EXERCÍCIO 5: FUNCIONALIDADE ADICIONAL")
# Sua implementação aqui...

### **7. Resumo do Módulo**

Vamos recapitular o que aprendemos:

In [ ]:
# �� RESUMO DO MÓDULO 6: RETRIEVAL

resumo = {
    "conceitos_principais": [
        "Retrieval é a parte do RAG que encontra informações relevantes",
        "Similaridade semântica é fundamental para bons resultados",
        "Diferentes estratégias podem ser combinadas para melhor performance",
        "Avaliação de qualidade é essencial para otimizar o sistema"
    ],
    "estrategias_aprendidas": [
        "Similarity Search - Busca por similaridade de embeddings",
        "Keyword Search - Busca por palavras-chave",
        "Multi-Query Retrieval - Múltiplas queries relacionadas",
        "Re-ranking - Melhorar ordem dos resultados",
        "Hybrid Retrieval - Combinar diferentes estratégias"
    ],
    "metricas_importantes": [
        "Precision - Proporção de resultados relevantes",
        "Recall - Proporção de documentos relevantes encontrados",
        "F1-Score - Média harmônica entre precision e recall",
        "Similarity Score - Score de similaridade semântica"
    ],
    "boas_praticas": [
        "Sempre teste múltiplas estratégias",
        "Use filtros para refinar resultados",
        "Implemente re-ranking para melhor qualidade",
        "Monitore métricas de qualidade",
        "Considere o contexto da aplicação"
    ]
}

print("�� RESUMO DO MÓDULO 6: RETRIEVAL")
print("=" * 50)

for categoria, itens in resumo.items():
    print(f"\n�� {categoria.replace('_', ' ').title()}:")
    for item in itens:
        print(f"   • {item}")

print("\n🚀 Próximo módulo: RAG Completo - Montando o Sistema!")
print("   Vamos juntar tudo e criar um sistema RAG completo!")

---

## **🎯 Conclusão do Módulo**

Neste módulo, aprendemos que **Retrieval** é como ter um detetive super inteligente que encontra exatamente as informações que você precisa. Não é apenas buscar, mas buscar de forma inteligente e eficiente.

**Principais takeaways:**
- ✅ Similaridade semântica é mais importante que palavras exatas
- ✅ Múltiplas estratégias podem ser combinadas
- ✅ Re-ranking melhora significativamente a qualidade
- ✅ Avaliação contínua é essencial

**Próximo passo:** No módulo 7, vamos aprender sobre **RAG Completo** - como juntar todos os componentes e criar um sistema funcional!

---

*💡 **Dica**: Experimente diferentes estratégias de retrieval com seus próprios documentos para encontrar a combinação ideal para seu caso de uso.*